In [1]:
import face_recognition
import cv2
import numpy as np

In [2]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

In [3]:
# Load a sample picture and learn how to recognize it.
remy_image = face_recognition.load_image_file("remy.jpg")
remy_face_encoding = face_recognition.face_encodings(remy_image)[0]

In [4]:
print(remy_face_encoding)

[-6.74583465e-02 -6.60402998e-02  2.42562983e-02 -9.65260807e-03
 -3.89938056e-02 -3.64935920e-02  5.52540570e-02 -2.78785434e-02
  1.61573842e-01 -2.15796549e-02  2.19327658e-01 -2.46821065e-03
 -2.17961907e-01 -4.52499427e-02 -7.81779438e-02  2.72037834e-02
 -1.40940562e-01 -9.95025560e-02 -4.22619209e-02 -1.03104256e-01
  6.52135760e-02  1.26903672e-02 -1.84537526e-02 -4.66367379e-02
 -1.56566560e-01 -3.11120033e-01 -1.37378201e-01 -1.78490788e-01
  2.13636160e-02 -4.11714129e-02  1.05857618e-01  5.76869212e-02
 -8.66422132e-02 -9.99242440e-03  5.55143207e-02  1.14922952e-02
  1.73792476e-04 -4.81524467e-02  1.97116673e-01  2.65083909e-02
 -9.75823626e-02 -1.78871397e-03  8.28818232e-02  2.98181534e-01
  9.45829228e-02  1.05857320e-01  7.91583769e-03 -3.55920792e-02
  9.93830934e-02 -2.43011057e-01  9.10205841e-02  1.88409612e-01
  5.93509525e-02  7.63706863e-02  1.60760656e-01 -1.34464845e-01
  5.06507196e-02  9.46052521e-02 -2.32672408e-01  1.00808203e-01
  4.33401391e-02  8.68024

In [5]:
cody_image = face_recognition.load_image_file("cody.jpg")
cody_face_encoding = face_recognition.face_encodings(cody_image)[0]

In [6]:
taylor_image = face_recognition.load_image_file("taylor.jpg")
taylor_face_encoding = face_recognition.face_encodings(taylor_image)[0]

In [7]:
# Create arrays of known face encodings and their names
known_face_encodings = [
    remy_face_encoding,
    cody_face_encoding,
    taylor_face_encoding
]
known_face_names = [
    "Remy",
    "Cody",
    "taylor"
]

In [8]:
print(cody_face_encoding)

[-7.15263328e-03  1.07421115e-01  2.11272333e-02 -4.10189442e-02
 -1.35224670e-01  3.02238241e-02 -7.52124339e-02 -1.01473089e-02
  2.30083629e-01 -3.04139331e-02  2.04139709e-01  4.49312031e-02
 -3.26619655e-01  1.91239305e-02 -6.15920573e-02  1.12495199e-01
 -2.27479130e-01 -4.71524559e-02 -7.00758100e-02 -7.94766918e-02
  9.95257050e-02  1.06244929e-01  9.14436653e-02  1.97496172e-02
 -1.03642426e-01 -3.79210442e-01 -9.85681713e-02 -9.59089696e-02
  9.83978137e-02 -1.61637172e-01  1.25847133e-02 -1.08352368e-04
 -1.40913442e-01 -1.58359744e-02  8.58172961e-03  3.06643825e-02
 -2.50899792e-02 -1.39891386e-01  2.17190266e-01  7.27810860e-02
 -1.64086744e-01  1.63341239e-02 -1.50181614e-02  2.36148730e-01
  1.24549493e-01  8.14079270e-02  4.08306643e-02 -1.02127157e-01
  1.10652424e-01 -2.16367275e-01  1.06705777e-01  2.17850804e-01
  1.23370312e-01  2.38958746e-02  3.41554172e-02 -1.52273446e-01
  2.03637872e-02  2.12212950e-01 -2.38599762e-01  7.93558061e-02
  1.37047455e-01 -1.02101

In [9]:
def start_cap():
    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            if len(face_encodings) > 0:
                for face_encoding in face_encodings:
                    # See if the face is a match for the known face(s)
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                    name = "Unknown"

                    # # If a match was found in known_face_encodings, just use the first one.
                    # if True in matches:
                    #     first_match_index = matches.index(True)
                    #     name = known_face_names[first_match_index]

                    # Or instead, use the known face with the smallest distance to the new face
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = known_face_names[best_match_index]

                    face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
start_cap()